# 03_Model_Training.ipynb

## Descripción
En esta fase se implementa el ciclo de entrenamiento del modelo **ConvNeXt Base**. Se utiliza aprendizaje por transferencia (Transfer Learning) desde pesos de ImageNet, adaptando la arquitectura para clasificación Multi-Label (Sigmoid).

## Características del Entrenamiento
* **Arquitectura:** ConvNeXt Base (88M Parámetros).
* **Optimizador:** AdamW (SOTA para Transformers/ConvNeXt).
* **Regularización:** MixUp (Data Augmentation) + Label Smoothing implícito.
* **Precisión:** Mixed Precision (FP16) para aceleración en GPU y optimización de VRAM.
* **Tracking:** Integración completa con MLflow para registro de métricas y artefactos.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, applications, optimizers, mixed_precision
import numpy as np
import matplotlib.pyplot as plt
import mlflow
import mlflow.tensorflow
from datetime import datetime
import os
import gc

# Limpieza de memoria y sesion
tf.keras.backend.clear_session()
gc.collect()

# Configuracion de Mixed Precision (FP16)
# Acelera el entrenamiento y reduce el uso de memoria VRAM sin perder precision significativa
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Constantes de Entrenamiento
IMG_SIZE = 224
BATCH_SIZE = 8          # Ajustado para GPU de 6-8GB VRAM
EPOCHS = 40
NUM_CLASSES = 3
MIXUP_ALPHA = 0.2       # Intensidad de la mezcla de imagenes

# Reproducibilidad
tf.random.set_seed(42)
np.random.seed(42)

# Verificacion de hardware
print(f"Politica de Precision: {policy.name}")
print(f"GPUs Disponibles: {len(tf.config.list_physical_devices('GPU'))}")

# Estructura de Directorios
BASE_DIR = '/tf/notebooks'
DATA_DIR = os.path.join(BASE_DIR, 'data')
MODELS_DIR = os.path.join(BASE_DIR, 'models')
LOGS_DIR = os.path.join(BASE_DIR, 'logs')
OUTPUTS_DIR = os.path.join(BASE_DIR, 'outputs')

for d in [MODELS_DIR, LOGS_DIR, OUTPUTS_DIR]:
    os.makedirs(d, exist_ok=True)

## 1. Carga de Datos Preprocesados
Se cargan los tensores `.npy` generados en el paso anterior. Se realiza un casteo explícito a `float32` para garantizar la compatibilidad con las operaciones de la GPU.

In [ ]:
# ============================================================================
# 1. CARGA DE DATASETS
# ============================================================================

print("Cargando tensores desde disco...")

X_train = np.load(os.path.join(DATA_DIR, 'X_train.npy')).astype('float32')
y_train = np.load(os.path.join(DATA_DIR, 'y_train.npy')).astype('float32')

X_val   = np.load(os.path.join(DATA_DIR, 'X_val.npy')).astype('float32')
y_val   = np.load(os.path.join(DATA_DIR, 'y_val.npy')).astype('float32')

X_test  = np.load(os.path.join(DATA_DIR, 'X_test.npy')).astype('float32')
y_test  = np.load(os.path.join(DATA_DIR, 'y_test.npy')).astype('float32')

print(f"Dimensiones de Entrenamiento: {X_train.shape}")
print(f"Dimensiones de Validacion:    {X_val.shape}")

## 2. Pipeline de Entrada (tf.data)
Se construye un pipeline de alto rendimiento utilizando `tf.data.Dataset`.
* **Resize:** Escalado bicúbico a 224x224.
* **MixUp:** Técnica de regularización que combina linealmente dos imágenes y sus etiquetas. Esto suaviza la superficie de decisión del modelo.
* **Prefetch:** Carga asíncrona de datos para evitar cuellos de botella en la GPU.

In [ ]:
# ============================================================================
# 2. CONFIGURACION DEL PIPELINE (ETL)
# ============================================================================

AUTO = tf.data.AUTOTUNE

def preprocess_image(image, label):
    """Redimensionado bicubico para mantener calidad visual"""
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE], method='bicubic')
    return image, label

def mixup(images, labels):
    """
    Implementacion vectorial de MixUp.
    Mezcla imagenes del mismo batch para regularizacion.
    """
    alpha = MIXUP_ALPHA
    batch_size = tf.shape(images)[0]
    
    # Generacion de indices aleatorios para mezclar
    indices = tf.random.shuffle(tf.range(batch_size))
    images_2 = tf.gather(images, indices)
    labels_2 = tf.gather(labels, indices)
    
    # Muestreo de la distribucion Beta
    gamma = tf.random.gamma(shape=[batch_size], alpha=alpha)
    beta = tf.random.gamma(shape=[batch_size], alpha=alpha)
    lam = gamma / (gamma + beta)
    lam = tf.reshape(lam, [-1, 1, 1, 1])
    
    # Mezcla lineal
    images_mix = images * lam + images_2 * (1 - lam)
    
    # Mezcla de etiquetas
    lam_labels = tf.reshape(lam, [-1, 1])
    labels_mix = labels * lam_labels + labels_2 * (1 - lam_labels)
    
    return images_mix, labels_mix

# Construccion de Datasets
ds_train = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(1000)
    .map(preprocess_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .map(mixup, num_parallel_calls=AUTO) # MixUp solo en entrenamiento
    .prefetch(AUTO)
)

ds_val = (
    tf.data.Dataset.from_tensor_slices((X_val, y_val))
    .map(preprocess_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

ds_test = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
    .map(preprocess_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

print("Pipeline de datos configurado y optimizado.")

## 3. Arquitectura del Modelo
Se instancia **ConvNeXt Base** con pesos pre-entrenados de ImageNet.
* **Backbone:** ConvNeXt Base (sin la capa superior).
* **Head (Clasificador):**
    * Global Average Pooling.
    * Layer Normalization (estándar para Transformers/ConvNeXt).
    * Dense 1024 (GELU) + Dropout.
    * **Salida:** 3 Neuronas con activación **Sigmoid** (Multi-label).

In [ ]:
# ============================================================================
# 3. DEFINICION DE LA ARQUITECTURA
# ============================================================================

def build_model():
    # Backbone pre-entrenado
    base_model = applications.ConvNeXtBase(
        include_top=False,
        weights='imagenet',
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_preprocessing=True # Normalizacion interna integrada
    )
    
    # Permitimos el fine-tuning completo
    base_model.trainable = True
    
    inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs)
    
    # Cabecera de clasificacion personalizada
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    
    x = layers.Dense(1024, activation='gelu')(x)
    x = layers.Dropout(0.3)(x)
    
    # Capa de salida (Sigmoid para independencia de clases)
    # dtype='float32' es necesario para estabilidad numerica con Mixed Precision
    outputs = layers.Dense(NUM_CLASSES, activation='sigmoid', dtype='float32', name='predictions')(x)
    
    return keras.Model(inputs=inputs, outputs=outputs, name="ConvNeXt_SOTA")

model = build_model()
# model.summary() # Descomentar para ver arquitectura completa

## 4. Configuración del Entrenamiento
* **Loss:** Binary Crossentropy (Estándar para multi-label).
* **Métricas:** Binary Accuracy y AUC (Area Under Curve).
* **Optimizador:** AdamW (Weight Decay desacoplado).
* **Callbacks:**
    * `ModelCheckpoint`: Guarda solo el mejor modelo.
    * `ReduceLROnPlateau`: Reduce el learning rate si la validación se estanca.
    * `EarlyStopping`: Detiene el entrenamiento si no hay mejora para evitar overfitting.

In [ ]:
# ============================================================================
# 4. COMPILACION Y CALLBACKS
# ============================================================================

# Optimizador AdamW (Recomendado para SOTA)
optimizer = optimizers.AdamW(learning_rate=5e-5, weight_decay=1e-4)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['binary_accuracy', keras.metrics.AUC(name='auc', multi_label=True)]
)

# Configuracion de MLflow
os.environ["MLFLOW_TRACKING_USERNAME"] = "admin"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "password1234"
mlflow.set_tracking_uri("http://mlflow:5000") # Ajustar URL segun entorno (http://mlflow:5000 interno)
mlflow.set_experiment("ConvNeXt_Production_V1")

# Callbacks
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint_path = os.path.join(MODELS_DIR, f'best_model_{timestamp}.keras')

callbacks_list = [
    callbacks.ModelCheckpoint(
        checkpoint_path, 
        monitor='val_binary_accuracy', 
        save_best_only=True, 
        mode='max', 
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.5, 
        patience=3, 
        verbose=1
    ),
    callbacks.EarlyStopping(
        monitor='val_binary_accuracy', 
        patience=8, 
        restore_best_weights=True, 
        verbose=1
    ),
    callbacks.CSVLogger(os.path.join(LOGS_DIR, f'training_log_{timestamp}.csv'))
]

print(f"Modelo configurado. Checkpoint: {checkpoint_path}")

## 5. Ejecución del Entrenamiento
Inicio del ciclo de entrenamiento con registro automático en MLflow.

In [ ]:
# ============================================================================
# 5. EJECUCION (TRAINING LOOP)
# ============================================================================

print("Iniciando entrenamiento...")

with mlflow.start_run(run_name=f"Train_{timestamp}") as run:
    # Registro de hiperparametros
    mlflow.log_params({
        "model": "ConvNeXt Base",
        "batch_size": BATCH_SIZE,
        "epochs": EPOCHS,
        "optimizer": "AdamW",
        "learning_rate": 5e-5
    })
    
    # Entrenamiento
    history = model.fit(
        ds_train,
        validation_data=ds_val,
        epochs=EPOCHS,
        callbacks=callbacks_list,
        verbose=1
    )
print("Entrenamiento finalizado exitosamente.")

## 6. Evaluación de Calidad del Modelo
Se genera el reporte de clasificación detallado y la matriz de confusión para detectar sesgos entre clases.

In [ ]:
# ============================================================================
# 6. EVALUACION Y METRICAS
# ============================================================================

print("Generando predicciones sobre Test Set...")
# Prediccion sobre todo el set de prueba
y_pred_probs = model.predict(ds_test)

# Convertir probabilidades a clases (Threshold 0.5 para logica binaria/multilabel)
# Para la matriz de confusion de CIFAR (que es single label), usamos argmax
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1) # y_test venia en one-hot

# 1. Reporte de Clasificacion
print("\n" + "="*60)
print("REPORTE DE CLASIFICACION")
print("="*60)
print(classification_report(y_true_classes, y_pred_classes, target_names=CLASS_NAMES))

# 2. Matriz de Confusion
cm = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.xlabel('Predicción')
plt.ylabel('Realidad')
plt.title(f'Matriz de Confusión (Accuracy Global: {history.history["val_binary_accuracy"][-1]*100:.2f}%)')
plt.savefig(os.path.join(OUTPUTS_DIR, 'confusion_matrix.png'))
plt.show()

# 3. Grafico de Entrenamiento
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss vs Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['binary_accuracy'], label='Train Accuracy')
plt.plot(history.history['val_binary_accuracy'], label='Val Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig(os.path.join(OUTPUTS_DIR, 'training_curves.png'))
plt.show()

## 7. Protocolo de Exportación a Producción (CPU-Safe)

Esta etapa implementa un procedimiento de exportación seguro diseñado para entornos de producción. Se realizan las siguientes operaciones críticas:

1.  **Aislamiento de Hardware (CPU Force):** Se fuerza al entorno a utilizar la CPU (`CUDA_VISIBLE_DEVICES="-1"`). Esto evita errores de fragmentación de memoria VRAM (OOM) al cargar simultáneamente el modelo de entrenamiento y la instancia de inferencia.
2.  **Stripping del Optimizador:** Se elimina el estado del optimizador (AdamW). Estos pesos son necesarios solo para el entrenamiento (retropropagación) pero inútiles para la inferencia. Eliminarlos reduce el tamaño del archivo final entre un 50% y un 70%.
3.  **Sanity Check:** Se realiza una carga de prueba y una predicción con datos sintéticos (dummy) para garantizar la integridad del archivo binario antes del despliegue.

In [ ]:
# ============================================================================
# 7. EXPORTACION SEGURA Y OPTIMIZACION
# ============================================================================

import os
import glob
import time
import tensorflow as tf
from tensorflow import keras
import numpy as np

# CONFIGURACION DE ENTORNO
# Forzamos el uso de CPU para liberar la GPU y evitar conflictos de memoria (OOM)
# durante la manipulacion final del modelo.
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

print("=" * 80)
print("INICIANDO PROTOCOLO DE EXPORTACION (MODO CPU)")
print("=" * 80)

# Directorios
BASE_DIR = '/tf/notebooks'
MODELS_DIR = os.path.join(BASE_DIR, 'models')
EXPORT_DIR = os.path.join(BASE_DIR, 'production_models')
os.makedirs(EXPORT_DIR, exist_ok=True)

# ----------------------------------------------------------------------------
# 1. Identificacion del mejor checkpoint
# ----------------------------------------------------------------------------
print("\nBuscando ultimo modelo entrenado...")
list_of_models = glob.glob(os.path.join(MODELS_DIR, 'best_model_*.keras'))

if not list_of_models:
    raise FileNotFoundError("No se encontraron modelos en la carpeta models/")

# Seleccionamos el mas reciente por fecha de modificacion
latest_model_path = max(list_of_models, key=os.path.getctime)
initial_size = os.path.getsize(latest_model_path) / (1024 * 1024) # MB

print(f"Modelo seleccionado: {os.path.basename(latest_model_path)}")
print(f"Peso original (con optimizador): {initial_size:.2f} MB")

# ----------------------------------------------------------------------------
# 2. Carga y Limpieza
# ----------------------------------------------------------------------------
print("Cargando modelo en memoria RAM (System Memory)...")
model = keras.models.load_model(latest_model_path)

export_path = os.path.join(EXPORT_DIR, 'model_convnext_base_final.keras')

print(f"Exportando version optimizada a: {export_path}")

# include_optimizer=False: Elimina los pesos de AdamW (reduccion ~60-70% size)
model.save(export_path, include_optimizer=False)

# Calculo de metricas de optimizacion
final_size = os.path.getsize(export_path) / (1024 * 1024)
reduction = (1 - (final_size / initial_size)) * 100

print("\nRESULTADOS DE LA OPTIMIZACION:")
print(f"Peso Inicial: {initial_size:.2f} MB")
print(f"Peso Final:   {final_size:.2f} MB")
print(f"Reduccion:    {reduction:.1f}%")

# ----------------------------------------------------------------------------
# 3. Verificacion de Integridad (Sanity Check)
# ----------------------------------------------------------------------------
print("\nValidando integridad del archivo exportado...")

# Liberar memoria del modelo pesado
del model
keras.backend.clear_session()

try:
    # Recarga del modelo ligero
    model_light = keras.models.load_model(export_path)
    print("El modelo se cargo correctamente.")

    # Inferencia de prueba con tensor de ceros
    dummy_input = tf.random.uniform((1, 224, 224, 3))
    print("Ejecutando prediccion de prueba...")
    
    start_time = time.time()
    pred = model_light.predict(dummy_input, verbose=0)
    duration = (time.time() - start_time) * 1000
    
    print(f"Prediccion exitosa. Shape de salida: {pred.shape}")
    print(f"Tiempo de inferencia (Cold Start): {duration:.2f} ms")
    
except Exception as e:
    print(f"ERROR CRITICO durante la validacion: {e}")
    raise e

print("=" * 80)
print("PROCESO FINALIZADO: Modelo listo para despliegue en API.")
print("=" * 80)